**Fin 585R**  
**Diether**  
**Problem Set**  
**Introduction to Portfolios**  
**Application: Short Selling**

**Overview**

This problem set is designed to introduce you to portfolio construction using a real portfolio application from the Finance literature. This assignment builds on the concepts and code I introduced last time in class. I think you'll find the Intro to Portfolios' Jupyter notebook helpful for this assignment.


**Short Selling Background**

When someone shorts a stock, they profit if the price of the stock goes down instead of going up, but short-selling transactions are more complicated than going long (buying a stock and then later selling it).  There are four basic steps to short selling:<br>

1. *The short seller borrows the desired number of shares from someone.* This is usually done by the broker who locates the shares and the broker becomes the middleman for the short seller and the lender (note, the broker is often both the middleman and the lender). The lender expects to be paid interest on the loan which is the main cost of shorting. The loan is callable by the lender at any time. The short seller can repay the loan at any time. <br><br>

2. *The short seller sells the shares.* The proceeds are put into an interest-bearing account called the collateral account. Most lenders require the collateral account to contain 102% of the value of the proceeds.  The collateral account usually invests in low risk, short term securities (e.g., Treasury bills). When the short seller borrows the stock there are lending fees; the short seller pays interest on the loan. Typically, the interest rate is small. The overall interest rate earned on the collateral account is split between the lender and the short seller. The portion of the interest rate received by the short seller is called the rebate rate. The **loan fee** is the portion paid the lender, and is equivalent to the interest rate the short seller pays on the loan. Therefore, the **loan fee** is the main direct cost of shorting. There can be a zero or negative rebate rate; a negative rebate rate corresponds to a situation where the lender receives all the interest in the collateral account and the short sellers pays additional interest out of her pocket to the lender.<br><br>

3. *Pay any dividends while the loan is open.* The short seller must pay to the lender the cash equivalent of any dividends paid out on the stock. If you short Apple, and Apple pays a 2 dollar dividend per share during the time you short the stock, then you owe the lender 2 dollars for every share you shorted. <br><br>

4. *Buy the shares back.* The short sellers profits are the following:

$$
Profit = Sell - Buy -(Interest \ Paid)
$$ <br> 
  
The Finance literature has been interested in whether short sellers have good information. Do short sellers typically make money when the short? If loan fees are high, then short sellers are paying a high price to short (bet that the stock will perform poorly). Therefore, if short sellers are paying a lot to short, it likely represents times when short sellers have good information. 

In this problem set you create portfolios based on lagged loan fee to test the preceding hypothesis. Remember, the returns in the data are from going long (buying and then selling the stock). Your portfolios will reflect that fact. If the returns are really low or negative for a portfolio, then that means the short sellers are experiencing high returns.


**Data**

The data are monthly stock data for all stocks in the U.S. with non-missing loan fee data. The basic unit of observation is the stock month. You can download the data directly using the following link: [the data](http://diether.org/prephd/03-mstk_short_02-12.csv). There is also a link on *Learning Suite*. The data contain the following variables:

|Variable | Description                                       |
|---------|---------------------------------------------------|
|permno   | stock identifier                                  |
|caldt    | calendar date                                     |
|ret      | monthly return                                    |
|prclag   | stock price, lagged                               |   
|melag    | market equity, lagged                             |
|feelag   | the loan fee expressed a percent per anum, lagged |

**Tasks and Questions**  

1. What is the sample period of this data?<br><br>

2. There are some observations where `feelag` is less than zero. These are data errors. Remove these observation from the dataframe and then compute the summary statistics for `feelag`.<br><br>

3. Output the median and 90th percentile of `feelag` for every month. This can be done in seperate commands. <br><br>

4. Construct three equal-weight portfolios using `feelag` a the criterion variable. Portfolio 0: an equal-weight portfolio the includes all stocks with `feelag` less than or equal to 3% (loan fee are express as the interest rate per anum). Portfolio 1: an equal-weight portfolio the includes all stocks with `feelag` greater than 3% and less than or eqal to 5%. Portfolio 2: an equal-weight portfolio the includes stocks with `feelag` greater than 5% (loan fee are express as the interest rate per anum). Hint: use the pd.cut to bin the data and use my Jupyter notebook from last class as a guide.<br><br>

5. Compute summary statistics for each of the portfolios you created including the mean, standard deviation, and t-statistics testing of the mean return of the portfolio is zero.<br><br>

5. Are your results consistent with the hypothesis that short sellers have good information?

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("03-mstk_short_02-12.csv",parse_dates=['caldt'])
df.head()

**Q1.** sample period of this data

In [ ]:
df['caldt'].min()

In [ ]:
df['caldt'].max()

**Q2.** Remove observation where feelag < 0

In [ ]:
df['feelag'].describe()

In [ ]:
df = df.query("feelag > -0.000001").reset_index(drop=True)
df

In [ ]:
df['feelag'].describe()

**Q3.** 90th percentile of `feelag` for every month

In [ ]:
df.groupby('caldt')['feelag'].quantile([0.5,0.9])

In [ ]:
df.groupby('caldt')['feelag'].quantile([0.5,0.9]).unstack(level=1)

In [ ]:
df.groupby('caldt')['feelag'].quantile([0.5,0.9]).unstack(level=1).plot()

**Q4. Form feelag based portfolios**

In [ ]:
df['bins'] = pd.cut(df['feelag'],[-0.01,3,5,100],labels=False)
df.head()

In [ ]:
port = df.groupby(['caldt','bins'])['ret'].mean()*100
port = port.unstack(level='bins')
port.head()

**Q5. Portfolio Summary Statistics**

In [ ]:
from finance_byu.summarize import summary
summary(port).loc[['count','mean','std','tstat','pval']].round(3)

**Q6. Do short sellers have good information?**

Are short sellers acting on good information that predicts declines in security prices? If so, securities with high short selling demand should have low next period returns. Right now (given what we've covered so far), we can't really tell. We haven't talked or modeled what the expected return on stocks should be and why (risk, the answer is risk). We can tell that the high lagged loan fee has the lowest average return; that's certainly consistent with the hypothesis. But that average isn't significant below zero (t-stat = -0.91). Remember, these are normal (long/buy) portfolios. The `high` portfolio is constructed as if the investor bought it. A negative average return means the high short costs predict low future average returns which is consistent with short-sellers picking stocks that go down on average

However, we can sharpen up our testing by thinking of a simple strategy where where we short 1000 dollars worth of portfolio `high` and then use the proceeds to invest 1000 dollars in portfolio `low`. This is called a pairs trade, and the return on the new portfolio is the following:

$$
r_{spread} = r_{low} - r_{high}
$$

In [ ]:
port['spread'] = port[0] - port[2]

In [ ]:
summary(port).loc[['count','mean','std','tstat','pval']].round(3)